# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. ___________________
2. __________________
3. ____________________
4. ___________________
5. ___________________

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

In [3]:
#Coding begins here

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block

Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below

In [17]:
#Code Block : Set Initial State (Must handle dynamic inputs)

import heapq
import math
from collections import deque
import time

def set_initial_state():
    # This function sets up the grid. 'O' means open path, 'B' means building, 'R' means roadblock. Here start and end positions are not
    # considered as they are the part of dynamic input section.
    # Example grid
    city_grid = [
        ['O', 'O', 'R', 'R', 'O', 'O'],
        ['B', 'O', 'R', 'O', 'O', 'O'],
        ['O', 'O', 'O', 'O', 'R', 'O'],
        ['B', 'O', 'R', 'O', 'O', 'B'],
        ['O', 'O', 'O', 'O', 'B', 'O'],
        ['O', 'B', 'O', 'R', 'O', 'O']
    ]
    return city_grid

In [18]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)

# Heuristic function (Euclidean distance + penalties)
def heuristic_function(current_position, goal_position):
    """
    Calculate the heuristic cost from the current position to the goal.
    Uses Euclidean distance as the heuristic.
    """
    x1, y1 = current_position
    x2, y2 = goal_position
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def calculate_movement_cost(current_position, neighbor_position):
    """
    Calculate the movement cost between the current position and a neighboring position.
    Includes a penalty for diagonal movement.
    """
    x1, y1 = current_position
    x2, y2 = neighbor_position
    return 1 + DIAGONAL_PENALTY if x1 != x2 and y1 != y2 else 1

# Cost function to move to a neighboring node
def calculate_path_cost_breakdown(optimal_path, grid):
    """
    Calculate and provide a detailed breakdown of the costs associated with each step of the optimal path.
    This includes movement costs and adjustments for adjacency to buildings or roadblocks.
    """
    total_gn = 0
    path_cost_details = []
    
    for i in range(1, len(optimal_path)):
        current = optimal_path[i-1]
        next_node = optimal_path[i]
        
        # Calculate the movement cost
        move_cost = calculate_movement_cost(current, next_node)
        
        # Calculate adjustment for adjacency to buildings or roadblocks
        adjustment = 0
        x2, y2 = next_node
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x2 + dx, y2 + dy
            if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]):
                if grid[nx][ny] == 'B':
                    adjustment += ADJACENT_BUILDING_REWARD
                elif grid[nx][ny] == 'R':
                    adjustment += ADJACENT_ROADBLOCK_PENALTY
        
        step_cost = move_cost + adjustment
        total_gn += step_cost
        
        step_details = {
            'current': next_node,
            'movement_cost': move_cost,
            'adjustment': adjustment,
            'step_total_cost': step_cost,
            'cumulative_cost': total_gn
        }
        path_cost_details.append(step_details)
    
    return total_gn, path_cost_details

In [19]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented

# This is coded as part of the RBFS function in the next section.


In [20]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented

def is_goal_reached(current_position, goal_position):
    """
    Check if the current position is the goal position.
    """
    return current_position == goal_position

### 2.	Definition of Algorithm 1 (Mention the Name of the algorithm here)

In [21]:
#Code Block : Function for algorithm 1 implementation

# RBFS Algorithm Implementation

def recursive_best_first_search(current_position, goal_position, grid, f_limit, g, path, nodes_expanded, max_depth):
    """
    Recursive Best-First Search (RBFS) algorithm for finding the optimal path.
    Returns the optimal path, the cumulative cost g(n), the f(n) value, the number of nodes expanded, and the maximum depth reached.
    """
    if is_goal_reached(current_position, goal_position):
        h = heuristic_function(current_position, goal_position)
        fn = g + h
        return path, g, fn, nodes_expanded, max_depth  # Goal found
    
    successors = []
    x, y = current_position
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and grid[nx][ny] == 'O':
            move_cost = calculate_movement_cost(current_position, (nx, ny))
            new_g = g + move_cost
            h = heuristic_function((nx, ny), goal_position)
            f = new_g + h
            successors.append(((nx, ny), f, new_g, h))
    
    if not successors:
        return None, math.inf, None, nodes_expanded, max_depth  # Dead end
    
    # Sort successors by f value
    successors.sort(key=lambda x: x[1])
    
    while successors:
        best = successors[0]
        if best[1] > f_limit:
            return None, best[1], None, nodes_expanded, max_depth
        
        alternative_f = successors[1][1] if len(successors) > 1 else math.inf
        
        result, best_f, fn, nodes_expanded, max_depth = recursive_best_first_search(
            best[0], goal_position, grid, min(f_limit, alternative_f), best[2], 
            path + [best[0]], nodes_expanded + 1, max(max_depth, len(path))
        )
    # Using the second best f-score for the backtracking process best_f
        successors[0] = (best[0], best_f, best[2], best[3])
        
        if result is not None:
            return result, best_f, fn, nodes_expanded, max_depth
        
        successors.sort(key=lambda x: x[1])
    
    return None, math.inf, None, nodes_expanded, max_depth


### 3.	Definition of Algorithm 2 (Mention the Name of the algorithm here)

In [22]:
#Code Block : Function for algorithm 2 implementation

### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [31]:
#Code Block : Function & call to get inputs (start/end state)

def get_dynamic_inputs():
    """
    Provide dynamic start and goal positions.
    These positions can be modified as needed for different test cases.
    """
    start_position = (0, 0)  # Example start position
    goal_position = (4, 3)   # Example goal position
    return start_position, goal_position

### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [32]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))

def find_and_evaluate_optimal_path():
    """
    Wrapper function to find the optimal path using the RBFS algorithm, and then calculate and print a detailed cost breakdown.
    """
    grid = setup_city_grid()
    
    # Get dynamic start and goal positions
    start, goal = get_dynamic_inputs()
    
    # Measure time complexity: start time
    start_time = time.time()
    
    # Step 1: Find the optimal path using RBFS
    path, _, total_fn, nodes_expanded, max_depth = recursive_best_first_search(
        start, goal, grid, math.inf, 0, [start], 1, 1
    )
    
    # Measure time complexity: end time
    end_time = time.time()
    time_taken = end_time - start_time
    
    if path:
        # Step 2: Calculate and print the cost breakdown for the optimal path
        final_gn, path_cost_details = calculate_path_cost_breakdown(path, grid)
        
        print("Optimal Path:", path)
        print("Total Cost of Optimal Path (g(n)):", final_gn)
        print("\nCost Breakdown for Optimal Path:")
        for step in path_cost_details:
            print(f"Step: {step['current']}, "
                  f"Movement Cost: {step['movement_cost']}, "
                  f"Adjustment: {step['adjustment']}, "
                  f"Step Total Cost: {step['step_total_cost']}, "
                  f"Cumulative Cost (g(n)): {step['cumulative_cost']}")
    else:
        print("No path found.")
    
    # Output additional details
    print("Nodes Expanded:", nodes_expanded)
    print("Space Complexity (Max Depth of Recursion):", max_depth)
    print(f"Time Complexity: {time_taken:.4f} seconds")


In [33]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))

### 5.	Comparitive Analysis

In [34]:
#Code Block : Print the Time & Space complexity of algorithm 1

find_and_evaluate_optimal_path()

# The output of this function will display the time complexity (execution time)
# and the space complexity (number of nodes expanded) for the RBFS algorithm.

Optimal Path: [(0, 0), (0, 1), (1, 1), (2, 1), (2, 2), (2, 3), (3, 3), (4, 3)]
Total Cost of Optimal Path (g(n)): 18

Cost Breakdown for Optimal Path:
Step: (0, 1), Movement Cost: 1, Adjustment: 3, Step Total Cost: 4, Cumulative Cost (g(n)): 4
Step: (1, 1), Movement Cost: 1, Adjustment: -2, Step Total Cost: -1, Cumulative Cost (g(n)): 3
Step: (2, 1), Movement Cost: 1, Adjustment: 0, Step Total Cost: 1, Cumulative Cost (g(n)): 4
Step: (2, 2), Movement Cost: 1, Adjustment: 6, Step Total Cost: 7, Cumulative Cost (g(n)): 11
Step: (2, 3), Movement Cost: 1, Adjustment: 3, Step Total Cost: 4, Cumulative Cost (g(n)): 15
Step: (3, 3), Movement Cost: 1, Adjustment: 3, Step Total Cost: 4, Cumulative Cost (g(n)): 19
Step: (4, 3), Movement Cost: 1, Adjustment: -2, Step Total Cost: -1, Cumulative Cost (g(n)): 18
Nodes Expanded: 9
Space Complexity (Max Depth of Recursion): 7
Time Complexity: 0.0001 seconds


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison : _______________________________________________

________________________________________________________

_________________________________________________________